![Egeria Logo](https://raw.githubusercontent.com/odpi/egeria/main/assets/img/ODPi_Egeria_Logo_color.png)

### Performance Suite Results
# Crux Plugin Repository Connector

## Calculating medians

This notebook simply calculates the medians for every method executed by the PTS.

## Setup

### Results locations

Locations for the results (see subdirectories in the same location where this notebook resides to review the raw results themselves):

In [ ]:
results = [
    "pts-05-02",
    "janus-05-02",
    "pts-10-05",
    "janus-10-05",
    "pts-20-10",
    "janus-20-10",
    "pts-40-10",
    "janus-40-10",
    "pts-80-10"
]

### Parsing and analysis methods

The following defines methods necessary to parse and process the results, and must be run prior to the subsequent cells.

In [ ]:
import os
import json
import pandas as pd
from IPython.display import display

def validateProfileResultsLocation(location):
    profile_details_location = location + os.path.sep + "profile-details"
    print("Validating profile-details location:", profile_details_location)
    if os.path.isdir(profile_details_location):
        print(" ... directory exists.")
    else:
        print(" ... ERROR: could not find this directory. Is the location specified correct?")

# Define the profile ordering
profile_order=[
    'Entity creation', 'Entity search', 'Relationship creation', 'Relationship search',
    'Entity classification', 'Classification search', 'Entity update', 'Relationship update',
    'Classification update', 'Entity undo', 'Relationship undo', 'Entity retrieval', 'Entity history retrieval',
    'Relationship retrieval', 'Relationship history retrieval', 'Entity history search', 'Relationship history search',
    'Graph queries', 'Graph history queries', 'Entity re-home', 'Relationship re-home', 'Entity declassify',
    'Entity re-type', 'Relationship re-type', 'Entity re-identify', 'Relationship re-identify',
    'Relationship delete', 'Entity delete', 'Entity restore', 'Relationship restore', 'Relationship purge',
    'Entity purge'
]

# Given a profileResult.requirementResults object, parse all of its positiveTestEvidence
# and group the results by methodName
def parseEvidence(df, repositoryName, requirementResults):
    if (requirementResults is not None and 'positiveTestEvidence' in requirementResults):
        print("Parsing evidence for:", requirementResults['name'], "(" + repositoryName + ")")
        data_array = []
        for evidence in requirementResults['positiveTestEvidence']:
            if ('methodName' in evidence and 'elapsedTime' in evidence):
                data = {
                    'repo': repositoryName,
                    'method_name': evidence['methodName'],
                    'elapsed_time': evidence['elapsedTime'],
                    'profile_name': requirementResults['name'],
                    'test_case_id': evidence['testCaseId'],
                    'assertion_id': evidence['assertionId']
                }
                data_array.append(data)
        df = df.append(pd.read_json(json.dumps(data_array), orient='records'), ignore_index=True)
    return df

# Given a profile detail JSON file, retrieve all of its profileResult.requirementResults[] objects
def parseRequirementResults(profileFile):
    with open(profileFile) as f:
        profile = json.load(f)
    # This first case covers files retrieved via API
    if ('profileResult' in profile and 'requirementResults' in profile['profileResult']):
        return profile['profileResult']['requirementResults']
    # This second case covers files created by the CLI client
    elif ('requirementResults' in profile):
        return profile['requirementResults']
    else:
        return None

# Retrieve a listing of all of the profile detail JSON files
def getAllProfiles(profileLocation):
    detailsLocation = profileLocation + os.path.sep + "profile-details"
    _, _, filenames = next(os.walk(detailsLocation))
    full_filenames = []
    for filename in filenames:
        full_filenames.append(detailsLocation + os.path.sep + filename)
    return full_filenames

# Parse all of the provided profile file's details into the provided dataframe
def parseProfileDetailsIntoDF(df, profileFile, qualifier):
    profileResults = parseRequirementResults(profileFile)
    if profileResults is not None:
        for result in profileResults:
            df = parseEvidence(df, qualifier, result)
    return df

# The results

## Parse all results

In [ ]:
df1 = pd.DataFrame({'repo': [], 'method_name': [], 'elapsed_time': [], 'profile_name': [], 'test_case_id': [], 'assertion_id': []})

for result_name in results:
    validateProfileResultsLocation(result_name)
    files = getAllProfiles(result_name)
    dfEnv = None
    for profile_file in files:
        df1 = parseProfileDetailsIntoDF(df1, profile_file, result_name)

## Print all results

In [ ]:
header  = "Profile | Method"
breaker = "---|---"
for result_name in results:
    header  += " | " + result_name
    breaker += "|---"
print(header)
print(breaker)
for profile_name in profile_order:
    groups = df1[df1['profile_name'] == profile_name].groupby(['repo', 'method_name'], as_index=False).median()
    methods = sorted(groups['method_name'].unique())
    grouped_results = []
    for result in results:
        grouped_results.append(groups[groups['repo'] == result])
    index = 0
    for method in methods:
        row = " ... | "
        if index == 0:
            row = profile_name + " | "
        row += method
        for result in grouped_results:
            value = result.loc[result['method_name'] == method, 'elapsed_time']
            if value.empty:
                row += " | --"
            else:
                row += " | " + str(value.values[0])
        print(row)
        index += 1

## Visualisation

Plot the results for visual comparison purposes.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from numpy import median

cruxOnly = [
    'Entity undo',
    'Relationship undo',
    'Entity history retrieval',
    'Relationship history retrieval',
    'Entity history search',
    'Relationship history search',
    'Graph queries',
    'Graph history queries'
]

alwaysSkip = [
    'Graph queries',
    'Graph history queries'
]

def compareResults(dfX, all_methods=False):
    if not all_methods:
        # Skip the profiles that are only implemented in the Crux repository
        dfX = dfX[~dfX['profile_name'].isin(cruxOnly)]
    else:
        dfX = dfX[~dfX['profile_name'].isin(alwaysSkip)]
    sns.set(font_scale=1.2)
    sns.set_style("whitegrid")
    method_order = []
    for profile_name in profile_order:
        if (all_methods and profile_name not in alwaysSkip) or profile_name not in cruxOnly:
            profile_details = df1[df1['profile_name'] == profile_name]
            for method_name in sorted(profile_details['method_name'].unique()):
                method_order.append(method_name)
    fix, axs = plt.subplots(ncols=1, nrows=1, figsize=(18,18))
    figure = None
    if all_methods:
        figure = sns.pointplot(x="elapsed_time", y="method_name", hue="repo",
                               data=dfX, estimator=median, ax=axs,
                               hue_order=sorted(results),
                               plot_kws=dict(alpha=0.5),
                               order=method_order)
    else:
        figure = sns.pointplot(x="elapsed_time", y="method_name", hue="repo",
                               data=dfX, estimator=median, ax=axs,
                               hue_order=sorted(results),
                               markers=['x', 'x', 'x', 'x', 'o', 'o', 'o', 'o', 'o'],
                               linestyles=['--', '--', '--', '--', '-', '-', '-', '-', '-'],
                               plot_kws=dict(alpha=0.5),
                               order=method_order)
    figure.set(xlabel='Elapsed time (ms)', ylabel='')
    figure.set_title("Typical execution time comparison")
    figure.get_legend().set(title="Repo and volume")
    plt.setp(figure.collections, alpha=0.5)
    display(fix)
    plt.tight_layout()
    plt.savefig('comparison.svg')
    plt.close(fix)

In [ ]:
compareResults(df1, all_methods=False)